# Basic Neural Net w/Keras

## Imports

In [155]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [156]:
%matplotlib inline
sns.set_style('darkgrid')

## Import data from Keras

In [157]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## Import necessary Keras functions

In [158]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D
from keras.utils import to_categorical
from keras.models import load_model
import keras

## Clean Data

In [159]:
n_train = x_train.shape[0]
n_test = x_test.shape[0]
n_rows = x_train.shape[1]
n_cols = x_train.shape[2]

n_classes = 10

# Cast to floats and normalize 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Reshape for Conv2D layers
x_train = x_train.reshape(n_train,n_rows,n_cols,1)
x_test = x_test.reshape(n_test,n_rows,n_cols,1)

# One Hot Encode categorical 
y_train = to_categorical(y_train,n_classes)
y_test = to_categorical(y_test,n_classes)

## Create and Fit NN

In [160]:
batch_size = 32
epochs = 10

model = Sequential()
model.add(Conv2D(filters=10,kernel_size=(3,3),activation='relu',input_shape=(n_rows,n_cols,1)))
model.add(Conv2D(filters=10,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(50,activation='relu'))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer='Adam',loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])

model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 48s 805us/step - loss: 0.2090 - accuracy: 0.9364 - val_loss: 0.0752 - val_accuracy: 0.9773
Epoch 2/10
60000/60000 [==============================] - 45s 757us/step - loss: 0.0780 - accuracy: 0.9760 - val_loss: 0.0468 - val_accuracy: 0.9843
Epoch 3/10
60000/60000 [==============================] - 46s 767us/step - loss: 0.0580 - accuracy: 0.9817 - val_loss: 0.0419 - val_accuracy: 0.9854
Epoch 4/10
60000/60000 [==============================] - 46s 765us/step - loss: 0.0452 - accuracy: 0.9851 - val_loss: 0.0374 - val_accuracy: 0.9877
Epoch 5/10
60000/60000 [==============================] - 46s 767us/step - loss: 0.0367 - accuracy: 0.9882 - val_loss: 0.0332 - val_accuracy: 0.9897
Epoch 6/10
60000/60000 [==============================] - 47s 781us/step - loss: 0.0324 - accuracy: 0.9893 - val_loss: 0.0361 - val_accuracy: 0.9888
Epoch 7/10
60000/60000 [==============================] 

## Get data for submission

In [164]:
dftest = pd.read_csv('./Data/test.csv')
test = dftest.values

# Cleaning again
test = test.astype('float32')
test /= 255
test = test.reshape(len(test),n_rows,n_cols,1)

## Make predictions

In [165]:
predictions = model.predict(test)
predictions = [np.argmax(x) for x in predictions]

# Create submission file

In [166]:
submission = pd.DataFrame(list(enumerate(predictions)),columns=['ImageId','Label']).set_index('ImageId')
submission.index += 1
submission.to_csv('./submission.csv')